# Import Required Libraries

In [2]:
from textwrap import dedent
from phi.assistant import Assistant
from phi.tools.serpapi_tools import SerpApiTools
from phi.tools.newspaper_toolkit import NewspaperToolkit
import streamlit as st
from phi.llm.openai import OpenAIChat
import logging

# Set up logging

In [3]:
logging.basicConfig(level=logging.INFO)

# Set up the Streamlit app

In [4]:
st.title("AI Journalist Agent 🗞️")
st.caption("Generate High-quality articles with AI Journalist by researching, writing, and editing quality articles on autopilot using GPT-4o")

2024-06-05 10:55:49.300 
  command:

    streamlit run C:\Users\Henry Morgan\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

# User authentication

In [6]:
st.subheader("User Authentication")
openai_api_key = st.text_input("Enter OpenAI API Key to access GPT-4o", type="password")
serp_api_key = st.text_input("Enter Serp API Key for Search functionality", type="password")
if openai_api_key and serp_api_key:
    try:
        # Initialize Assistants

        # Searcher Assistant
        searcher = Assistant(
            name="Searcher",
            role="Searches for top URLs based on a topic",
            llm=OpenAIChat(model="gpt-4o", api_key=openai_api_key),
            description=dedent(
                """\
                You are a world-class journalist for the New York Times. Given a topic, generate a list of 3 search terms
                for writing an article on that topic. Then search the web for each term, analyze the results
                and return the 10 most relevant URLs.
                """
            ),
            instructions=[
                "Given a topic, first generate a list of 3 search terms related to that topic.",
                "For each search term, `search_google` and analyze the results.",
                "From the results of all searches, return the 10 most relevant URLs to the topic.",
                "Remember: you are writing for the New York Times, so the quality of the sources is important.",
            ],
            tools=[SerpApiTools(api_key=serp_api_key)],
            add_datetime_to_instructions=True,
        )

        # Writer Assistant
        writer = Assistant(
            name="Writer",
            role="Retrieves text from URLs and writes a high-quality article",
            llm=OpenAIChat(model="gpt-4o", api_key=openai_api_key),
            description=dedent(
                """\
                You are a senior writer for the New York Times. Given a topic and a list of URLs,
                your goal is to write a high-quality NYT-worthy article on the topic.
                """
            ),
            instructions=[
                "Given a topic and a list of URLs, first read the article using `get_article_text`.",
                "Then write a high-quality NYT-worthy article on the topic.",
                "The article should be well-structured, informative, and engaging.",
                "Ensure the length is at least as long as a NYT cover story -- at a minimum, 15 paragraphs.",
                "Ensure you provide a nuanced and balanced opinion, quoting facts where possible.",
                "Remember: you are writing for the New York Times, so the quality of the article is important.",
                "Focus on clarity, coherence, and overall quality.",
                "Never make up facts or plagiarize. Always provide proper attribution.",
            ],
            tools=[NewspaperToolkit()],
            add_datetime_to_instructions=True,
            add_chat_history_to_prompt=True,
            num_history_messages=3,
        )

        # Editor Assistant
        editor = Assistant(
            name="Editor",
            llm=OpenAIChat(model="gpt-4o", api_key=openai_api_key),
            team=[searcher, writer],
            description="You are a senior NYT editor. Given a topic, your goal is to write a NYT-worthy article.",
            instructions=[
                "Given a topic, ask the search journalist to search for the most relevant URLs for that topic.",
                "Then pass a description of the topic and URLs to the writer to get a draft of the article.",
                "Edit, proofread, and refine the article to ensure it meets the high standards of the New York Times.",
                "The article should be extremely articulate and well written.",
                "Focus on clarity, coherence, and overall quality.",
                "Ensure the article is engaging and informative.",
                "Remember: you are the final gatekeeper before the article is published.",
            ],
            add_datetime_to_instructions=True,
            # debug_mode=True,
            markdown=True,
        )

        # Input field for the report query
        st.subheader("Article Generation")
        query = st.text_input("What do you want the AI journalist to write an Article on?")
        length = st.selectbox("Select the article length", options=["Short (5-10 paragraphs)", "Medium (10-15 paragraphs)", "Long (15+ paragraphs)"])
        tone = st.selectbox("Select the tone of the article", options=["Neutral", "Positive", "Negative", "Informative", "Opinionated"])

        if query:
            with st.spinner("Processing..."):
                try:
                    # Get the response from the assistant
                    response = editor.run(query, stream=False)
                    st.write(response)
                except Exception as e:
                    logging.error("Error generating article: %s", e)
                    st.error("An error occurred while generating the article. Please try again.")

    except Exception as e:
        logging.error("Error initializing assistants: %s", e)
        st.error("An error occurred while setting up the assistants. Please check your API keys and try again.")
else:
    st.info("Please enter both OpenAI and Serp API keys to proceed.")

### Detailed Explanation:
1. Import Required Libraries
The required libraries are imported, including those for creating AI assistants, searching the web, retrieving article text, and setting up the Streamlit web application.

2. Set Up Logging
Logging is set up to capture and display any errors that occur during the execution of the app.

3. Set Up Streamlit App
The title and description of the Streamlit app are set.

4. User Authentication
Input fields are created for the user to enter their OpenAI and SerpAPI keys. These keys are required to access the OpenAI GPT-4 model and perform web searches.

5. Initialize Assistants
Three assistants are initialized:

Searcher: Generates search terms, performs web searches, and returns relevant URLs.
Writer: Retrieves text from the URLs and writes a high-quality article.
Editor: Coordinates the search and writing tasks, edits and refines the article to ensure it meets high standards.
6. Input Field for Article Query
Additional input fields are provided for the user to specify the topic, length, and tone of the article they want to generate.

7. Process the Query and Generate the Article
When the user enters a query and specifies the article requirements, the editor assistant coordinates the search and writing tasks to generate the final article. Progress indicators are displayed while the article is being processed, and any errors that occur are logged and displayed to the user.

Key Features:
- User Authentication: Ensures that only authenticated users can use the app by requiring API keys.
- Enhanced Input Fields: Captures detailed requirements for the article, such as length and tone.
- Logging and Monitoring: Keeps track of errors and other important events for debugging and monitoring.
- Error Handling: Gracefully handles errors and informs the user.
- Progress Indicators: Keeps the user informed about the current state of the process.
- Customization Options: Allows users to customize the length and tone of the article, providing more control over the output.